<a href="https://colab.research.google.com/github/Aidzillafont/Video-Games-/blob/main/Video_Games_NoteBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration


In [1]:
import pandas as pd
df = pd.read_csv('https://github.com/Aidzillafont/Video-Games-/blob/843e8c4c47db94fc39b083f82226d4b88c8924a1/vgsales.csv?raw=true')
df.dtypes

Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object

In [2]:
#check for Nas in the columns
df.isna().any(axis=0)
#we can see year and publisher are missing values

Rank            False
Name            False
Platform        False
Year             True
Genre           False
Publisher        True
NA_Sales        False
EU_Sales        False
JP_Sales        False
Other_Sales     False
Global_Sales    False
dtype: bool

In [3]:
#since there is no meaning fully way to impute these values we will remove them into another df for now
df_nona = df.dropna(axis=0)
df_nona.isna().any(axis=0)

Rank            False
Name            False
Platform        False
Year            False
Genre           False
Publisher       False
NA_Sales        False
EU_Sales        False
JP_Sales        False
Other_Sales     False
Global_Sales    False
dtype: bool

Lets check the genres and see how they rank on global sales

In [14]:
#lets check the best selling genre of all time
best_genre_df = df.iloc[:,4:].groupby(['Genre']).sum().sort_values(['Global_Sales'], ascending=False)
best_genre_df

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Genre,,,,,
Action,877.83,525.00,159.95,187.38,1751.18
Sports,683.35,376.85,135.37,134.97,1330.93
Shooter,582.60,313.27,38.28,102.69,1037.37
Role-Playing,327.28,188.06,352.31,59.61,927.37
Platform,447.05,201.63,130.77,51.59,831.37
Misc,410.24,215.98,107.76,75.32,809.96
Racing,359.42,238.39,56.69,77.27,732.04
Fighting,223.59,101.32,87.35,36.68,448.91
Simulation,183.31,113.38,63.70,31.52,392.20


Here we can see that Action sells the most and Strategy sells the least. Lets visualise this

In [31]:
import plotly.express as px

fig = px.bar(best_genre_df,  x=['NA_Sales',	'EU_Sales',	'JP_Sales',	'Other_Sales'], y=best_genre_df.index, title="Best Genre by Sales", text_auto=True, width=1250)
#to reverse order so action is at the top
fig.update_yaxes(autorange="reversed")
fig.show()

From the above visualisation we can see not only the best genre but how they sell in each region. Something to note here is North America is responsible for most sales in every genre bar Role-Playing. The best region to sell role playing is Japan according to the data.

So we know the best genre for games per region but what consoles are these games on and who makes them? Ares specific genres being targeted by specific consoles or publishers. Lets take a look with another visulaisation!

But first we are gonna need to construct a dataframe to examine this! We will take a look a genre by platform first

In [90]:
df_genre_by_platform = df[['Genre', 'Platform','Rank']].groupby(['Genre', 'Platform']).count()
#reseting the index here so we can use column names in our plotly sunburst
df_genre_by_platform.reset_index(inplace=True)
#rename rank to represent what it really is count of games on that platform
df_genre_by_platform.rename(columns={'Rank':'# of Games'}, inplace=True)

fig = px.sunburst(df_genre_by_platform, path=['Genre', 'Platform'], values='# of Games', width=500, title='Sunburst of Genres and Platforms')
fig.show()

Oh no! This graph looks a little squishy. Sunburts can be a great visualization tool but it has a downside being when you have too many categories then it becomes all squished together and loses some of its elegabilty.  
Luckely for us plotly is interactive so if you check out my python notebook you can click into the graph yourself and zoom in to categories. If you clicked Action you would see something a little like this...  
maybey not as colourful though. 

In [84]:
fig = px.sunburst(df_genre_by_platform[df_genre_by_platform['Genre']=='Action'], path=['Genre', 'Platform'], values='# of Games', width=500)
fig.show()

Lets try and get a less squishy view of everything by using a bubble chart. In the bubble chart below the size of the bubble is is gong to be the number of games on that platform genre pair.

In [80]:
fig = px.scatter(df_genre_by_platform, x='Platform', y='Genre',
	         size='# of Games', color='Platform', width=1500, title='Number of Games on Platform Genre Pairs')
#to reverse order so action is at the top
fig.update_yaxes(autorange="reversed")
fig.show()

Cool so the bubble lays it out with more clarity but you loose all number of games proportionality that you with a sunburst or pie chart.

Thats interesting there seems to be some platforms where the really is not alot of games on for example the PCFX only has 1 game! 

Its also worth noting that some platforms really dont cater to some game genre for example all the playstation platforms dont really have many puzzle games.

So with the combination of the bubble and sunburst we can see that most genres are dominates by only a handful of platforms and some platforms really have most of the games. Lets visualise that second part in a sunburst

In [91]:
fig = px.sunburst(df_genre_by_platform, path=['Platform','Genre'], values='# of Games', width=500, title='Sunburst of Genres and Platforms')
fig.show()

From the above we can see nearly half of all games are on only 5 platforms.

So lets summarise what we have found:

*   The best selling games are Action games 
*   Only 5 platforms account for half of all the games
  * Those platforms being DS, PS2, PS3, Wii and X360
* Different platforms cater more to different genres
  * for example DS has the most simulation, puzzle and misc games

This is for the number of games on platforms what about sales?

In [97]:
df_gp_sales = df[['Genre', 'Platform', 'Global_Sales']].groupby(['Genre', 'Platform']).sum()
#reseting the index here so we can use column names in our plotly sunburst
df_gp_sales.reset_index(inplace=True)

fig = px.sunburst(df_gp_sales, path=['Genre', 'Platform'], values='Global_Sales', width=500, title='Sunburst of Genres and Platforms by sales')
fig.show()

fig = px.sunburst(df_gp_sales, path=['Platform', 'Genre'], values='Global_Sales', width=500, title='Sunburst of Genres and Platforms by sales')
fig.show()

As can be seen from the above the effect we saw with number of games become even more pronounced. 

Lets finally look of the best games what genre were they and what platform did they sell on

In [109]:
df.loc[df.groupby(['Genre']).Rank.idxmin()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
16,17,Grand Theft Auto V,PS3,2013.0,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
50,51,Super Mario Land 2: 6 Golden Coins,GB,1992.0,Adventure,Nintendo,6.16,2.04,2.69,0.29,11.18
39,40,Super Smash Bros. Brawl,Wii,2008.0,Fighting,Nintendo,6.75,2.61,2.66,1.02,13.04
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
5,6,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
9,10,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31
10,11,Nintendogs,DS,2005.0,Simulation,Nintendo,9.07,11.00,1.93,2.75,24.76


So the best selling action game sold on PS3 but the best selling shooter did not sell on X360 the platform with the most shooter games and shooter games sold. 

Those are the best what about the average?

In [122]:
df_avg = df[['Genre','Platform', 'Global_Sales']].groupby(['Genre','Platform']).mean()
df_avg.reset_index(inplace=True)
df_avg.loc[df_avg.groupby(['Genre']).Global_Sales.idxmax()]

,Genre,Platform,Global_Sales
9,Action,NES,2.211538
34,Adventure,NES,4.380000
58,Fighting,NES,1.635000
82,Misc,NES,1.795000
108,Platform,NES,3.420714
128,Puzzle,GB,3.164667
167,Racing,WiiU,2.590000
174,Role-Playing,GB,4.201905
206,Shooter,NES,5.088571
230,Simulation,N64,1.019000


Why does the platforms with the most games and most sales not appear here to have the best average sales?

This is likely due to the fact there is really so many more games on the larger platforms so people pick and choose the games they buy. So on average there is less sales per game.

As for the NES of course it has the best average since there is no where near as many games on that platform. Comparing it to the PS3 that has over 300+ Action games where the NES only has 14.